## Basic formula

In [59]:
from collections import namedtuple
import math
from math import sqrt
import numpy as np

import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)
    

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [60]:
def checkcar(var):
    if isinstance(var, tuple):
        print(f'        converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print(f'        converting {var} into polar to calculate with')
        return car2pol(var)
    if isinstance(var, int):
        print(f'        int found for {var} assuming angle = 0 for polar calculation')
        return (var, 0)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print(f'    current in polar notation: {car2pol(cur)}')
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print(f'    voltage in polar notation: {car2pol(vol)}')
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi

def rad2deg(rad):
    return rad / 2 / math.pi * 360


def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'    Q = {voltage[0]:.2f} * {current[0]:.2f} * sin({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -Q')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    # print(f'    U = {voltage}, I = {current}')
    print(f'    P = {voltage[0]:.2f} * {current[0]:.2f} * cos({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -P')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [61]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

## Solver for complex equations

In [62]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

### Sandbox

In [63]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()





# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.864750979992415, -35.21359368159654)
(15.879783371533687, 85.75016285047698)
(26.466305619222812, -4.249837149523023)


In [64]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

## Transformers

In [65]:
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    if isinstance(current, complex): # it's in carthesian, convert to polar
        current = car2pol(current)
    if isinstance(current, tuple): # it's in polar, ignore the phase angle
        current = current[0]
    
    p = current ** 2 * resistance
    print(f'    using I2* R: {current}**2 {resistance} = {p}')
    return Power(p,0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    print(f'Total reactance for source {res}')
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Pprim {P(Uprim, Iprim)}W Qprim {Q(Uprim, Iprim)}VAR')
    psource = -P(primaryvoltage, Iprim)
    qsource = -Q(primaryvoltage, Iprim)
    print(f'Powers on primary side Psource: {psource}, Qsource {qsource}')
    print(f'Resistances on primary side: {power(Iprim, primaryresistance)}')
    print(f'Resistances on secondary side: {power(Isec, secondaryresistance)}')
    print(f'Efficiency transformer: {power(Isec, secondaryresistance).Watt} / {psource} = {power(Isec, secondaryresistance).Watt / psource}')

In [66]:
def nullast(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan parallel aan de bron dus P = U**2 / R gebruiken
    Rij = U**2 / P  
    Xmu = U**2 / Q
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Nullast:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f'Rij = {Rij} : U**2 / P\n'
        f'Xmu = {Xmu} : U**2 / Q\n')
    return Rij, Xmu


def kortsluit(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan in serie met bron dus P = I**2 * R gebruiken
    Rk = P / I**2
    Xk = Q / I**2
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Kortsluit:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f"Rk = {Rk} : P / I ** 2. Note Rk = R1 + R'2\n"
        f'Xk = {Xk} : Q / I ** 2\n')
    return Rk, Xk



### Sandbox

## Asynchronous machines

In [67]:
from math import sqrt
def phase2linevoltage(phasevoltage):
    return sqrt(3) * phasevoltage
def line2phasevoltage(linevoltage):
    return linevoltage / sqrt(3)
line2phasevoltage(10000)

def S3f(Ul, I):
    return sqrt(3) * Ul * I

def Pdriehoek(Pster):
    return 3 * Pster

In [68]:
def slip(fnet, frotor):
    # slip = 1, means rotor is stationary
    # slip = 0, means fnet == frotor
    # slip < 0, means machine is acting as generator
    return (fnet - frotor) / fnet


def slip2freq(slip, fnet=None):
    if not fnet: fnet = 50
    # slip = (fnet - frot) / fnet
    return -(slip * fnet - fnet)


In [69]:
def R2(Rk, ohmmeasured):
    # vervangingsweerstand as berekenen bij bekend zijn van Rk vanuit kortsluitproef
    return Rk - (ohmmeasured / 2)

def Ras(R2, slip):
    # vervangingsweerstand as berekenen inclusief slip
    return R2 * (1-slip) / slip

def Pas(I, R2, slip):
    # asvermogen bepalen
    return 3 * I**2 * Ras(R2, slip)

def koppel(P, freq):
    # P = w T
    return P / (freq * 2 * math.pi)

In [70]:
def Ras2slip(Ras, R2):
    R = Ras / R2
    return 1 / (R + 1)



In [71]:
def asynchrone(voltage, Xk, Rk, R2, slip):
    # gegeven vervangingsschema en een slip bepaal stroom, vermogens en rendement
    # returns asvermogen
    if isinstance(voltage, int):
        voltage = (voltage, 0)
    reactance = complex(Rk + Ras(R2, slip), Xk)
    cur = checkpol(current(voltage, reactance))
    print('Asynchronous machine')
    print('Total reactance = ', reactance)
    print(f'Current = {cur} (voltage / reactance)')
    print('P = ', -3* P(voltage, cur))
    print('Q = ', -3*Q(voltage, cur))
    print('Pas = ', Pas(cur[0], R2, slip))
    print('rendement bij slip > 0 (anders 1/x doen)', Pas(cur[0], R2, slip) / (3 * P(voltage, cur)))
    return Pas(cur[0], R2, slip)

### Sandbox

## Synchnonous machines

In [72]:


# def tas(slip, fnet):
#     was = 2 * math.pi * fnet * (1-slip)
#     print(slip)
#     return p(slip) / was
# s = np.linspace(-0.99,0.99,20)
# fnet = 50
# t = [tas(x, fnet) for x in s]

In [73]:
def X1(Unom, I2nullast, I1kortsluit, I2kortsluit):
    lamb = Unom / I2nullast
    print(f'Bij een I2 van {I2nullast}A wordt de nominale fasespanning gehaald van {Unom}V')
    I1nullast = I1kortsluit / I2kortsluit * I2nullast
    print(f'Bij een I2 van {I2kortsluit}A treedt een I1 op van {I1kortsluit}A, dus bij I2 van {I2nullast}A is I1 {I1nullast}A')
    X1 = Unom / I1nullast
    print('Hieruit volgt:')
    print(f'X1 = {X1} Ohm ({Unom} / {I1nullast})')
    print(f'Lambda = {lamb} ({Unom} / {I2nullast})')
    return X1

X1(230, 4,70, 12)

Bij een I2 van 4A wordt de nominale fasespanning gehaald van 230V
Bij een I2 van 12A treedt een I1 op van 70A, dus bij I2 van 4A is I1 23.333333333333332A
Hieruit volgt:
X1 = 9.857142857142858 Ohm (230 / 23.333333333333332)
Lambda = 57.5 (230 / 4)


9.857142857142858

In [74]:
# capacitive behavior = phase angle current > phase angle voltage, like a balloon, where flow goes before size
# inductive behavior = phase angle voltage > current, like a train where voltage goes before motion

# if I net phase angle < 0: angle in machine diagram goes right up (inductive behavior of net), this is correct since voltage angle > current

In [75]:
def behavior(phi):
    phi = (phi + 360) % 360
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    return angle_induccapa, angle_motorgenerator

def cosphi_to_options(cosphi):
    # if you don't know if cosphi is for Inet or I1, just enter the cos phi and it will explain how it behaves
    phi = math.acos(cosphi) / 2 / math.pi * 360
    print(f'phi = {phi}, but could also have been {-phi} or with 180 degrees offset if we looked the other way')
    for p in (phi, -phi):
        for reverse in (0,180):
            newphi =  p + reverse
            print(f'{newphi} leads to {behavior(newphi)}')

In [76]:
def syn_net(U1, I1, X1, l = None):
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = {U12} = Unet - UX1 --> {U1} - {car2pol(UX1)}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    print(QX1, Qas, machineQ)
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    assert induccapa == angle_induccapa and motorgenerator == angle_motorgenerator
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12


In [77]:
def syn_island(resistance_island, U1, X1, l = None):
    Iz = car2pol(current(U1, resistance_island)) # calculate Iz from U1 and resistance
    I1 = (Iz[0], (Iz[1] + 180) % 360) # I1 == Iz, but reverse phase angle
    print(f'I1 = {I1}')
    
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = {U12} = Unet - UX1 --> {U1} - {car2pol(UX1)}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    assert behavior(phi) == (induccapa, motorgenerator) 
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12


### Sandbox

In [78]:
syn_net((230,0), (54.35,217), 14)

        converting (54.35, 217) into carthesian to calculate with
    voltage in polar notation: (760.9000000000001, -53.00000000000002)
Voltage over X1 (760.9000000000001, -53.00000000000002)
U12 = (649.0185871656213, 110.55938395147095) = Unet - UX1 --> (230, 0) - (760.9000000000001, -53.00000000000002)
    P = 649.02 * 54.35 * cos(110.56 - 217.00) WARNING, if current enters at - should be -P
Pas is -29950.02958003861
    Q = 649.02 * 54.35 * sin(110.56 - 217.00) WARNING, if current enters at - should be -Q
Qas = -101495.77890926346
QX1 = 124064.745 (3 * I**2 * X1)
    P = 230.00 * 54.35 * cos(0.00 - 217.00) WARNING, if current enters at - should be -P
Pnet = 29950.02958003856
    Q = 230.00 * 54.35 * sin(0.00 - 217.00) WARNING, if current enters at - should be -Q
Qnet = -22568.96609073653
124064.745 -101495.77890926346 22568.966090736532
machine is operating 22568.966090736532, therefore inductive
lasthoek delta = 110.55938395147095, therefore generator behavior
asserting behavior p

(649.0185871656213, 110.55938395147095)

In [79]:
cosphi_to_options(0.8)

phi = 36.86989764584401, but could also have been -36.86989764584401 or with 180 degrees offset if we looked the other way
36.86989764584401 leads to ('capacitive', 'motor')
216.86989764584402 leads to ('inductive', 'generator')
-36.86989764584401 leads to ('inductive', 'motor')
143.13010235415598 leads to ('capacitive', 'generator')


In [80]:
syn_net((6060,0), (6870,143), 0.75)

        converting (6870, 143) into carthesian to calculate with
    voltage in polar notation: (5152.5, -127.00000000000004)
Voltage over X1 (5152.5, -127.00000000000004)
U12 = (10042.618252244087, 24.189172669857317) = Unet - UX1 --> (6060, 0) - (5152.5, -127.00000000000004)
    P = 10042.62 * 6870.00 * cos(24.19 - 143.00) WARNING, if current enters at - should be -P
Pas is -99746859.84417278
    Q = 10042.62 * 6870.00 * sin(24.19 - 143.00) WARNING, if current enters at - should be -Q
Qas = -181357675.22061208
QX1 = 106193025.0 (3 * I**2 * X1)
    P = 6060.00 * 6870.00 * cos(0.00 - 143.00) WARNING, if current enters at - should be -P
Pnet = 99746859.84417269
    Q = 6060.00 * 6870.00 * sin(0.00 - 143.00) WARNING, if current enters at - should be -Q
Qnet = 75164650.22061215
106193025.0 -181357675.22061208 -75164650.22061208
machine is operating -75164650.22061208, therefore capacitive
lasthoek delta = 24.189172669857317, therefore generator behavior
asserting behavior phase angle for 

(10042.618252244087, 24.189172669857317)

In [81]:
syn_net((6060,0), (6870,143), 0.75)

        converting (6870, 143) into carthesian to calculate with
    voltage in polar notation: (5152.5, -127.00000000000004)
Voltage over X1 (5152.5, -127.00000000000004)
U12 = (10042.618252244087, 24.189172669857317) = Unet - UX1 --> (6060, 0) - (5152.5, -127.00000000000004)
    P = 10042.62 * 6870.00 * cos(24.19 - 143.00) WARNING, if current enters at - should be -P
Pas is -99746859.84417278
    Q = 10042.62 * 6870.00 * sin(24.19 - 143.00) WARNING, if current enters at - should be -Q
Qas = -181357675.22061208
QX1 = 106193025.0 (3 * I**2 * X1)
    P = 6060.00 * 6870.00 * cos(0.00 - 143.00) WARNING, if current enters at - should be -P
Pnet = 99746859.84417269
    Q = 6060.00 * 6870.00 * sin(0.00 - 143.00) WARNING, if current enters at - should be -Q
Qnet = 75164650.22061215
106193025.0 -181357675.22061208 -75164650.22061208
machine is operating -75164650.22061208, therefore capacitive
lasthoek delta = 24.189172669857317, therefore generator behavior
asserting behavior phase angle for 

(10042.618252244087, 24.189172669857317)

## Kortsluit

In [82]:
# Helpers

# P = S * cosphi
# Q = S * sinphi

def cosphitosin(cosphi):
    return sqrt(1-cosphi**2)

def powertocosphi(P, S):
    return P / S

### Generators

In [83]:
def gen_current(Sn, Uline):
    # can be used for nominal calculations (Unline) but also for actual calculations (Uline)
    # assumes input is line voltage
    return Sn / (sqrt(3) * Uline)

def gen_reactance(xd, Ulinenom, Sn):
    return xd * Ulinenom **2 / Sn

def gen_kortsluitstroom(U12, Xg):
    return U12 / Xg

### Cables

In [154]:
def cable_impedance(R, X, km):
    # assumes R and X are given per km and are not expressed in milliHenry or something
    # If you don't know X but know L you need to multiply with omaga (314)
    # print(X)
    return complex(R * km, X * km)

def cable_admitantie(C, km):
    # de admitantie is calculated based on bedrijfscapaciteit
    bedrijfscapaciteit = C * km # in F
    return bedrijfscapaciteit * 314 

def admitantie_to_reactance(Y):
    # we split admitantie to a left and right side and calculate it into an impedance (by 1/Y)
    Y /= 2
    return complex(0,-1/Y)
    
def cable_capacitive_current(C, km, Ufase):
    bedrijfscapaciteit = C * km
    capacitieve_stroom = 314 * bedrijfscapaciteit * Ufase
    return capacitieve_stroom



### Transformers

In [85]:
def trans_impedance(uk, U3f, S, P):
    # 0 <= uk <= 1
    Z = uk * U3f**2 / S
    R = P * U3f**2 / S**2
    X = sqrt(Z**2 - R**2)
    print(f'Zt {Z} \nRt {R} \nXt {X}')
    return complex(R, X)

def trans_kortsluit(Rk, Ifase, Unom3f):
    Uk = Rk * Ifase
    u = Uk / Unom3f / sqrt(3)
    print()
    return u
 

### Grids

In [146]:
def grid_impedance(Sk, Unomlijn, c=1):
    return complex(0, c * Unomlijn**2 / Sk)

def grid_kortsluitstroom(Sk, Unomlijn):
    # je zou ook Sk / 3 kunnen doen is geleverde vermogen per fase. En dat vervolgens delen door Unomfase    
    return Sk / (sqrt(3)*Unomlijn)

def grid_Sk(Ik, Unomlijn):
    return sqrt(3) * Ik * Unomlijn

grid_kortsluitstroom(400*10**6, 110000)




2099.455524325912

### Loads

In [155]:
def load_Z(Unomlijn, S):
    # doesn't do anything with cosphi, for that you need to plug this into load_impedance as R
    # this formula is useful if you don't know R but do know S
    return Unomlijn**2 / S

def load_impedance(R, cosphi):
    return complex(R * cosphi, R * cosphitosin(cosphi))

### Overall

In [133]:
def ZnettoRX(factor, Znet):
    # sqrt((factor * X)**2 + X**2) == Znet
    divider = sqrt(1 + factor ** 2)
    print(divider)
    X = Znet / divider
    R = factor * X
    print(f'R: {R}\nX: {X}')
ZnettoRX(0.1, 0.22)

def middenspanningsnet(U3f, km, Ccable, Rcable, Xcable, Rload, cosphi):
    Xcable = cable_impedance(Rcable, Xcable, km)
    adm = cable_admitantie(Ccable, km)
    XCbhalf = admitantie_to_reactance(adm)
    Xload = load_impedance(Rload, cosphi)
    print(f'Xcable {Xcable} \nXCbhalf {XCbhalf} \nXload {Xload}')
    
    Zparallel = parallel(XCbhalf, Xload)
    vervangingsweerstand = Xcable + Zparallel
    print('par', parallel(XCbhalf, Xload))
    print(f'vervangingsweerstand {vervangingsweerstand}')
    

    I = current(U3f/sqrt(3), vervangingsweerstand)
    print(f'I {I}')
    
    Ulast1f = voltage(I, Zparallel)
    print(f'Ulast1f {car2pol(Ulast1f)}, dit is {car2pol(Ulast1f)[0] / (10000/sqrt(3))} tov dan de nominale waarde')
    return I
middenspanningsnet(10500, 50, 400*10**-9, 0.1, 450*10**-6*314, 10000000000000000, 0.82)
    
    
    
def vermogens(U3f, impedances, c=1):
    U1f = U3f / sqrt(3)
    I1f = current(c * U1f, sum(impedances))
    print(f'I1f, {car2pol(I1f)} with total impedance {sum(impedances)}')
    print(f'Calculation is I = U / R, where U == U3f / sqrt(3)  * c, and R is total impedance')
    for i in impedances:
        v = voltage(I1f, i)
        p = 3 * P(v, I1f)
        q = 3 * Q(v, I1f)
        s = S(p,q)
        print(f'\nfor impedance {i}:\nvoltage = {v}\nP = {p} W\nQ = {q} var\nS = {s} VA\n')


1.004987562112089
R: 0.021890818184619763
X: 0.21890818184619762
Xcable (5+7.065j) 
XCbhalf -318.4713375796178j 
Xload (8200000000000000+5723635208501675j)
par (8.312832000000302e-12-318.4713375796236j)
vervangingsweerstand (5.0000000000083125-311.4063375796236j)
    current in polar notation: (19.464589596906023, 89.08012699616786)
I (0.3124869139610104+19.462081083599085j)
    voltage in polar notation: (6198.913884365088, -0.9198730038306505)
Ulast1f (6198.913884365088, -0.9198730038306505), dit is 1.0736833799464476 tov dan de nominale waarde


In [134]:
c = cable_impedance(0.325, 0.12, 10)
t = trans_impedance(0.04, 10000, 250000, 4000)
l = load_impedance(1.2, 0.85) * (10000/400)**2
c,t,l

Zt 16.0 
Rt 6.4 
Xt 14.664242223858688


((3.25+1.2j), (6.4+14.664242223858688j), (637.5+395.0870157319777j))

### Sandbox

## Koppellijn

In [93]:
def line_power(U1, U2, Xlijn, delta):
    return U1 * U2 / Xlijn * math.sin(delta / 360 * 2 * math.pi)
line_power(400,400,1.5,-40)

def line_current(U1, U2, Xlijn, delta):
    p = line_power(U1, U2, Xlijn, delta)
    print('line power', p)
    return p / (U1 * math.cos(deg2rad(delta/2)))
    
line_current(400,400,1.5,-40)

line power -68564.01169989753


-182.4107431070233

In [94]:
def koppellijn(U2, I, angle, X): # U1, P1, P2, Q1, Q2
    Ux = voltage((I, angle), complex(0,X))
    print(f'Ux = {car2pol(Ux)}')
    U1 = pol2car(U2,0) + Ux
    print(f'U1 = {car2pol(U1)} --> Ux + U2')
    P1 = -P(U1, (I, angle))
    print(f'P1 = {3*P1}')
    P2 = P((U2,0), (I, angle))
    print(f'P2 = {3*P2}')
    Q1 = -Q(U1, (I, angle))
    print(f'Q1 = {3*Q1}')
    Q2 = Q((U2,0), (I, angle))
    print(f'Q2 = {3*Q2}')
    Qx = Q(Ux, (I, angle))
    print(f'Qx = {3*Qx}')


### Sandbox

In [95]:
fasespanning = 10500/sqrt(3)
fasespanning

6062.177826491071

In [96]:
koppellijn(fasespanning, 250, 30, 1)

        converting (250, 30) into carthesian to calculate with
    voltage in polar notation: (250.0, 120.0)
Ux = (250.0, 120.0)
U1 = (5941.124097624728, 2.0884343166034762) --> Ux + U2
        converting (5937.177826491071+216.50635094610968j) into polar to calculate with
    P = 5941.12 * 250.00 * cos(2.09 - 30.00) WARNING, if current enters at - should be -P
P1 = -3937500.000000001
    P = 6062.18 * 250.00 * cos(0.00 - 30.00) WARNING, if current enters at - should be -P
P2 = 3937500.000000001
        converting (5937.177826491071+216.50635094610968j) into polar to calculate with
    Q = 5941.12 * 250.00 * sin(2.09 - 30.00) WARNING, if current enters at - should be -Q
Q1 = 2085816.6849341518
    Q = 6062.18 * 250.00 * sin(0.00 - 30.00) WARNING, if current enters at - should be -Q
Q2 = -2273316.6849341514
        converting (-124.99999999999999+216.50635094610968j) into polar to calculate with
    Q = 250.00 * 250.00 * sin(120.00 - 30.00) WARNING, if current enters at - should be -Q
Q

In [97]:
sqrt(3937500**2 + 2273316**2)


4546633.027401266

In [98]:
51961.524227066315/3

17320.508075688773

In [99]:
line_current(400,400,1.5,-40)

line power -68564.01169989753


-182.4107431070233

## Primaire regeling

In [100]:
def generator_constant(pnom, fnom, s):
    # returns generator constant in MW/Hz
    return pnom / fnom * 100 / s

def area_constant(generator_constant, num_generators):
    # returns a Knet in MW/Hz
    return generator_constant * num_generators

def freq_change(delta_p, knet):
    # returns frequency change in Hz
    return -delta_p / knet

def primaire_regeling(freq_change, kgen):
    # returns change in power in MW
    return -kgen * freq_change

def secundaire_regeling(p_werkelijk, p_programma, ka, freq_change):
    return (p_werkelijk - p_programma) / ka + freq_change





In [101]:
generator_constant(500,50,4),generator_constant(600,50,6),generator_constant(200,50,5),generator_constant(250,50,5)

(250.0, 200.0, 80.0, 100.0)

In [102]:
generator_constant(500,50,4),generator_constant(540,50,6),generator_constant(400,50,5),generator_constant(400,50,5)

(250.0, 180.0, 160.0, 160.0)

In [103]:
250+ 200 + 80 + 100 + 250 + 180 + 160

1220

In [104]:
generator_constant(500,50, 4), generator_constant(600,50, 4)


(250.0, 300.0)

In [105]:
area_constant(250,3), area_constant(300,3), 

(750, 900)

In [106]:
df = freq_change(300,1220)

In [107]:
for k in [250, 200 , 80 , 100 , 250 , 180 ,160]:
    print(primaire_regeling(df, k))

61.47540983606557
49.18032786885246
19.672131147540984
24.59016393442623
61.47540983606557
44.26229508196721
39.34426229508197


In [108]:
361+399+220+175

1155

In [109]:
461+444+239

1144

### Sandbox

In [110]:
600-155

445